In [1]:
import numpy as np
import pandas as pd
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2 #260x260
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential
import imgaug.augmenters as iaa
import imgaug

from sklearn.model_selection import train_test_split

2023-01-19 12:37:58.199575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'
seed = 42

In [3]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_train_full = pd.concat([df_train_full, df_extra], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,4675,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,0875,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,4436,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8265,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [4]:
image_width = 260
image_height = 260
NUM_CLASSES = 6
batch_size = 32
n_epochs = 50
learning_rate=3e-5

In [5]:
df_train, df_val, y_train, y_val = train_test_split(df_train_full, df_train_full['label'], test_size = 0.2, shuffle = True, stratify = df_train_full['label'], random_state = seed)
df_train

,Id,label,filename
7592,eu.95a129ef-d94a-4ab0-a4e4-e66aac61e55f,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7704,eu.b90f170d-c2d2-47bf-bd61-375f9032e98f,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
7840,eu.84a95cb4-6026-40ed-bfb6-936d297a7d91,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
5030,5485,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7114,eu.edfa5512-6f5d-4cf7-bbcc-a31eace9b484,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
...,...,...,...
5033,5725,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4172,5120,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4507,9013,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
3625,2070,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [6]:
train_aug = iaa.Sequential(
    [
        iaa.Fliplr(0.3),
        iaa.Flipud(0.3), 
        iaa.Sometimes(0.1, iaa.Cutout()),
        iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5))),
    ],
    name = "train_aug"
)

In [7]:
def build_model(num_classes):
    
    inputs = layers.Input(shape = (image_width, image_height, 3))
    x=inputs
    #x = train_aug(inputs)
    model = EfficientNetB2(include_top=False, input_tensor=x, weights="imagenet")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [10]:
train_datagen = ImageDataGenerator(preprocessing_function=train_aug.augment_image)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.,
)

Found 6390 validated image filenames belonging to 6 classes.
Found 1598 validated image filenames belonging to 6 classes.


In [9]:
model = build_model(num_classes=NUM_CLASSES)

history = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=val_generator
)

2023-01-19 12:38:10.819220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 12:38:10.865016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 12:38:10.865245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 12:38:10.865866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/50


2023-01-19 12:38:18.532411: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


  1/200 [..............................] - ETA: 19:51 - loss: 2.7045 - accuracy: 0.2500

2023-01-19 12:38:19.395483: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 12:38:19.396033: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 12:38:19.396043: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-01-19 12:38:19.396692: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 12:38:19.396729: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-01-19 12:38:19.422369: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged onc

200/200 [==============================] - 133s 640ms/step - loss: 2.1357 - accuracy: 0.2734 - val_loss: 1.3357 - val_accuracy: 0.5131
Epoch 2/50
200/200 [==============================] - 137s 684ms/step - loss: 1.2270 - accuracy: 0.5604 - val_loss: 0.7966 - val_accuracy: 0.7541
Epoch 3/50
200/200 [==============================] - 124s 621ms/step - loss: 0.8515 - accuracy: 0.7141 - val_loss: 0.5485 - val_accuracy: 0.8367
Epoch 4/50
200/200 [==============================] - 126s 633ms/step - loss: 0.6431 - accuracy: 0.7879 - val_loss: 0.4233 - val_accuracy: 0.8686
Epoch 5/50
200/200 [==============================] - 127s 635ms/step - loss: 0.5198 - accuracy: 0.8330 - val_loss: 0.3517 - val_accuracy: 0.8892
Epoch 6/50
200/200 [==============================] - 134s 673ms/step - loss: 0.4492 - accuracy: 0.8549 - val_loss: 0.3071 - val_accuracy: 0.9036
Epoch 7/50
200/200 [==============================] - 130s 651ms/step - loss: 0.3999 - accuracy: 0.8681 - val_loss: 0.2750 - val_accura

In [11]:
df_test = pd.read_csv(data_dir+'/test.csv', dtype={'Id': str})
df_test['filename'] = data_dir+'/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,3962,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,9271,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,5133,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8842,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [12]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    shuffle=False
)

Found 3808 validated image filenames.


In [13]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 125s 1s/step


In [14]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [15]:
predictions = classes[y_pred.argmax(axis=1)]

In [16]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len(data_dir+'/images/'):-4]
del df_submission['filename']

In [17]:
df_submission[['Id', 'label']].to_csv('submissions/effb2_imnet_aug_v2.csv', index=False)

In [18]:
!kaggle competitions submit kitchenware-classification -f submissions/effb2_imnet_aug_v2.csv -m 'validation: 0.9537'

100%|██████████████████████████████████████| 38.8k/38.8k [00:02<00:00, 16.8kB/s]
Successfully submitted to Kitchenware Classification